# Download Meteor M2 satellite info and calculate times for recording

Requires an API key for n2yo.com and username for geonames.org

In [1]:
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.2.1 ──
✔ ggplot2 3.2.1     ✔ purrr   0.3.2
✔ tibble  2.1.3     ✔ dplyr   0.8.3
✔ tidyr   1.0.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.4.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


In [2]:
library(httr)

In [3]:
library(lubridate)


Attaching package: ‘lubridate’

The following object is masked from ‘package:base’:

    date



In [4]:
library(jsonlite)


Attaching package: ‘jsonlite’

The following object is masked from ‘package:purrr’:

    flatten



In [5]:
library(assertthat)


Attaching package: ‘assertthat’

The following object is masked from ‘package:tibble’:

    has_name



In [6]:
paste("Running at ", Sys.time() %>% .POSIXct("GMT"), "GMT")

[1] "Running at  2019-09-22 19:02:55 GMT"

# Helper functions

In [7]:
possibly_get_data <- function(url, n_tries, ...){

    rate <- rate_backoff(pause_base = 1, max_times = n_tries)
    possibly_insistent_get <- insistently(GET, rate, quiet = FALSE) %>% possibly(otherwise = NULL)

    possibly_insistent_get(url, ...)
}

In [8]:
assert_any_response <- function (x) assert_that(!is.null(x), 
                                                msg = paste("\nUnable to update data, is the network up?"))

In [9]:
assert_200_response <- function (x) assert_that(x$status_code >= 200 & x$status_code < 400,
                                                msg = paste("\nUnable to update data",
                                                            "\nstatus code:", x$status_code, 
                                                            "\nerror messge:", x$error))

# Config

What satellites are we trying to get orbital data for?

In [10]:
satellites_df <- tibble("satellite"=c("METEOR-M2", "METEOR-M2-2", "NOAA-15", "NOAA-18", "NOAA-19"),
                     "frequency" = c(137100000, 137100000, 137620000, 137912500, 137100000),
                     "norad_id" = c(40069, 44387, 25338, 28654, 28654),
                     "priority" = c(4,5,3,2,1))

In [11]:
satellites_df

satellite,frequency,norad_id,priority
<chr>,<dbl>,<dbl>,<dbl>
METEOR-M2,137100000,40069,4
METEOR-M2-2,137100000,44387,5
NOAA-15,137620000,25338,3
NOAA-18,137912500,28654,2
NOAA-19,137100000,28654,1


API key for n2yo.com

In [12]:
n2yo_api_key <- "&apiKey=CL4ZW2-LEYV8F-XRA2PC-46TQ"

Username for geonames.org

In [13]:
geonames_username <- "pgcudahy"

Minimal satellite elevation above the horizon before starting data capture (in **degrees**)

In [14]:
minimum_observable_elevation = 20

Station longitude, latitude. Positive values for E, negative for W

In [15]:
station_latitude <- c(-29.53)
station_longitude <- c(30.25)

Get the station's elevation (in meters above sea level)

In [16]:
station_elevation <- possibly_get_data(paste0("http://api.geonames.org/gtopo30JSON?lat=", station_latitude, 
                                   "&lng=", station_longitude, "&username=", geonames_username), 8)

In [17]:
station_elevation <- content(station_elevation)$gtopo30

Sanity check that your coordinates are close to a nearby city

In [18]:
nearby_cities <- possibly_get_data(paste0("http://api.geonames.org/findNearbyPlaceNameJSON?lat=", station_latitude, 
                                   "&lng=", station_longitude, "&username=", geonames_username), 8)

In [19]:
paste0("Station is located near ", content(nearby_cities)$geonames[[1]]$toponymName, ", ",
    content(nearby_cities)$geonames[[1]]$countryName, " at an elevation of ", station_elevation,
    " meters above sea level")

[1] "Station is located near Merrivale, South Africa at an elevation of 1167 meters above sea level"

In [20]:
localTimeZone <- Sys.timezone()

# Load previous satellite orbital data

In [21]:
satellite_data_df <- tryCatch({
    readRDS("satellite_data_df.rds")}, 
    warning = function(w) {
        print("No prior satellite orbital data found on disk")
        tibble(satellite=character(),
            startAz=double(),
            startAzCompass=character(),
            startUTC=as.POSIXct(character()),
            maxAz=double(),
            maxAzCompass=character(),
            maxEl=double(),
            maxUTC=integer(),
            endAz=double(),
            endAzCompass=character(),
            endUTC=as.POSIXct(character()),
            startDate=character(),
            startTime=character(),
            endDate=character(),
            endTime=character(),
            localStartTime=character(),
            duration=double())
}, error = function(e) {
        print("There was an error in trying to load satellite orbital data from disk")
        tibble(satellite=character(),
            startAz=double(),
            startAzCompass=character(),
            startUTC=as.POSIXct(character()),
            maxAz=double(),
            maxAzCompass=character(),
            maxEl=double(),
            maxUTC=integer(),
            endAz=double(),
            endAzCompass=character(),
            endUTC=as.POSIXct(character()),
            startDate=character(),
            startTime=character(),
            endDate=character(),
            endTime=character(),
            localStartTime=character(),
            duration=double())
    }
)

Get rid of passes that have already occurred

In [22]:
current_time_utc <- Sys.time() %>% .POSIXct("GMT")

In [23]:
satellite_data_df <- satellite_data_df %>% filter(startUTC > current_time_utc)

# Update satellite orbital data

##  Using the https://www.n2yo.com/api/ webservice since I can't find any R packages to predict satellite passes locally

Request: /radiopasses/{id}/{observer_lat}/{observer_lng}/{observer_alt}/{days}/{min_elevation}
```
Parameter       Type	Required	Comments
id              integer Yes	        NORAD id
observer_lat	float   Yes	        Observer's latitide (decimal degrees format)
observer_lng	float   Yes	        Observer's longitude (decimal degrees format)
observer_alt	float   Yes	        Observer's altitude above sea level in meters
days            integer Yes	        Number of days of prediction (max 10)
min_elevation   integer Yes	        The minimum elevation acceptable for the highest altitude point of the pass (degrees)
```

Generate API access urls for each satellite based on its NORAD ID

In [24]:
n2yo_urls <- satellites_df %>% pluck("norad_id") %>% paste("https://www.n2yo.com/rest/v1/satellite/radiopasses", .,
                            station_latitude, station_longitude, station_elevation, 10, minimum_observable_elevation, 
                            n2yo_api_key, sep="/")

Try to safely access the API

In [25]:
satellite_data <- n2yo_urls %>% map(possibly_get_data, 8) %>% set_names(satellites_df$satellite)

Test if the server responded

In [26]:
satellite_data %>% map(assert_any_response)

$`METEOR-M2`
[1] TRUE

$`METEOR-M2-2`
[1] TRUE

$`NOAA-15`
[1] TRUE

$`NOAA-18`
[1] TRUE

$`NOAA-19`
[1] TRUE

Test that the response was successful

In [27]:
satellite_data %>% map(assert_200_response)

$`METEOR-M2`
[1] TRUE

$`METEOR-M2-2`
[1] TRUE

$`NOAA-15`
[1] TRUE

$`NOAA-18`
[1] TRUE

$`NOAA-19`
[1] TRUE

Extract the payload from the API response

In [28]:
satellite_passes <- satellite_data %>% map(function (x) fromJSON(rawToChar(x$content)) %>% pluck("passes"))

Bind satellite info from the satellites_df dataframe to each row of the orbital data

In [29]:
new_satellite_data_df <- map2(names(satellite_passes), satellite_passes, 
                          function(x,y) cbind(satellite=x, y, stringsAsFactors=FALSE)) %>% 
                          bind_rows %>% full_join(satellites_df)

Joining, by = "satellite"


Format the start and stop times in UTC

In [30]:
new_satellite_data_df <- new_satellite_data_df %>% mutate(startUTC = as_datetime(startUTC, tz = "UTC")) %>%
    mutate(endUTC = as_datetime(endUTC, tz = "UTC"))

### Filter out passes that we've already downloaded so that they're not duplicated when we bind the new data to the old we've loaded from disk

In [31]:
new_satellite_data_df <- anti_join(new_satellite_data_df, satellite_data_df, by = c("satellite", "startUTC"))

For `mlrpt` need arguments in the form of `mlrpt $startTime-$stopTime -t $duration`. Start and stop time are in the format HHMM in **UTC**. Duration is how long the command needs to run (in **minutes**) from start to shut-down. Make it 3 minutes longer than the time of the satellite pass to give time for start-up, and at the end, decoding and saving images.

Also need the start time in **local** timezone to schedule `mlrpt` using `systemd`. Move it back 60 seconds to give `systemd` and `mlrpt` time to start up.

In [32]:
new_satellite_data_df <- new_satellite_data_df %>% 
    mutate(startDate = format(startUTC, "%Y-%m-%d")) %>%
    mutate(startTime = format(startUTC, "%H%M")) %>%
    mutate(endDate = format(endUTC, "%Y-%m-%d")) %>%
    mutate(endTime = format(endUTC, "%H%M")) %>%
    mutate(localStartTime = format(as_datetime(startUTC - 60, tz = localTimeZone), "%H:%M")) %>% 
    mutate(duration = as.numeric(ceiling(endUTC - startUTC)))

In [33]:
satellite_data_df <- bind_rows(satellite_data_df, new_satellite_data_df) %>% 
    select('satellite', 'frequency', 'norad_id', 'priority', everything()) %>%
    arrange(startDate, startTime)

In [34]:
saveRDS(satellite_data_df, "satellite_data_df.rds")